# Using Linear model

- Applying embedding



# 1)-Importing key Modules

In [0]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [2]:
! pip install dynet

     |████████████████████████████████| 28.1MB 1.4MB/s 


In [0]:
from collections import defaultdict
import math
import time
import random
import dynet as dy
import numpy as np

In [4]:
! pip install version_information

  Created wheel for version-information: filename=version_information-1.0.3-cp36-none-any.whl size=3880 sha256=7da10607b7db970da59015711308d7f3de7ba3657fd4ecf8a55d1b40c25af4be
  Stored in directory: /root/.cache/pip/wheels/1f/4c/b3/1976ac11dbd802723b564de1acaa453a72c36c95827e576321
Successfully built version-information


In [5]:
# first install: pip install version_information
%reload_ext version_information
%version_information pandas,torch,numpy

Software versions
Python 3.6.8 64bit [GCC 8.3.0]
IPython 5.5.0
OS Linux 4.14.137+ x86_64 with Ubuntu 18.04 bionic
pandas 0.25.3
torch 1.3.1+cu100
numpy 1.17.4
Sun Nov 17 19:52:48 2019 UTC

# 2)- Setting up neural network

In [0]:
N = 2 # The length of the n-gram
EMB_SIZE = 128 # The size of the embedding
HID_SIZE = 128 # The size of the hidden layer

In [0]:
# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
S = w2i["<s>"]
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      yield [w2i[x] for x in line.strip().split(" ")]

# 3)- Loading Data

### loading data using traditional format
using read()

In [0]:
train = list(read_dataset("train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("valid.txt"))
i2w = {v: k for k, v in w2i.items()}
nwords = len(w2i)

# 4)- Model Building

### 4.1)- Start DyNet and define trainer

In [0]:
model = dy.ParameterCollection()
trainer = dy.SimpleSGDTrainer(model, learning_rate=0.1)

### 4.2)-Define the model

In [0]:
W_emb = model.add_lookup_parameters((nwords, EMB_SIZE)) # Word weights at each position
W_h = model.add_parameters((HID_SIZE, EMB_SIZE * N))    # Weights of the softmax
b_h = model.add_parameters((HID_SIZE))                  # Weights of the softmax
W_sm = model.add_parameters((nwords, HID_SIZE))         # Weights of the softmax
b_sm = model.add_parameters((nwords))                   # Softmax bias

### 4.3)-A function to calculate scores for one value

In [0]:
def calc_score_of_history(words):
  # Lookup the embeddings and concatenate them
  emb = dy.concatenate([W_emb[x] for x in words])
  # Create the hidden layer
  h = dy.tanh(dy.affine_transform([b_h, W_h, emb]))
  # Calculate the score and return
  return dy.affine_transform([b_sm, W_sm, h])

### 4.4)-Calculate the loss value for the entire sentence

In [0]:
def calc_sent_loss(sent):
  # Create a computation graph
  dy.renew_cg()
  # The initial history is equal to end of sentence symbols
  hist = [S] * N
  # Step through the sentence, including the end of sentence token
  all_losses = []
  for next_word in sent + [S]:
    s = calc_score_of_history(hist)
    all_losses.append(dy.pickneglogsoftmax(s, next_word))
    hist = hist[1:] + [next_word]
  return dy.esum(all_losses)

### 4.5)-Generate a sentence

In [0]:
MAX_LEN = 100

In [0]:
def generate_sent():
  dy.renew_cg()
  hist = [S] * N
  sent = []
  while True:
    p = dy.softmax(calc_score_of_history(hist)).npvalue()
    next_word = np.random.choice(nwords, p=p/p.sum())
    if next_word == S or len(sent) == MAX_LEN:
      break
    sent.append(next_word)
    hist = hist[1:] + [next_word]
  return sent

# 5)- Train Model

In [16]:
for ITER in range(10):
  # Perform training
  random.shuffle(train)
  train_words, train_loss = 0, 0.0
  start = time.time()
  for sent_id, sent in enumerate(train):
    my_loss = calc_sent_loss(sent)
    train_loss += my_loss.value()
    train_words += len(sent)
    my_loss.backward()
    trainer.update()
    if (sent_id+1) % 5000 == 0:
      print("--finished %r sentences" % (sent_id+1))
  print("iter %r: train loss/word=%.4f, ppl=%.4f, time=%.2fs" % (ITER, train_loss/train_words, math.exp(train_loss/train_words), time.time()-start))

--finished 5000 sentences
--finished 10000 sentences
--finished 15000 sentences
--finished 20000 sentences
--finished 25000 sentences
--finished 30000 sentences
--finished 35000 sentences
--finished 40000 sentences
iter 0: train loss/word=5.8904, ppl=361.5453, time=1539.64s
--finished 5000 sentences
--finished 10000 sentences
--finished 15000 sentences
--finished 20000 sentences
--finished 25000 sentences
--finished 30000 sentences
--finished 35000 sentences
--finished 40000 sentences
iter 1: train loss/word=5.4269, ppl=227.4502, time=1534.92s
--finished 5000 sentences
--finished 10000 sentences
--finished 15000 sentences
--finished 20000 sentences
--finished 25000 sentences
--finished 30000 sentences
--finished 35000 sentences
--finished 40000 sentences
iter 2: train loss/word=5.2224, ppl=185.3853, time=1534.46s
--finished 5000 sentences
--finished 10000 sentences
--finished 15000 sentences
--finished 20000 sentences
--finished 25000 sentences
--finished 30000 sentences
--finished 350

KeyboardInterrupt: ignored

We Interrupted due to long computing

# 5)- Evaluate

In [19]:
 #Evaluate on dev set
dev_words, dev_loss = 0, 0.0
start = time.time()
for sent_id, sent in enumerate(dev):
  my_loss = calc_sent_loss(sent)
  dev_loss += my_loss.value()
  dev_words += len(sent)
print("iter %r: dev loss/word=%.4f, ppl=%.4f, time=%.2fs" % (ITER, dev_loss/dev_words, math.exp(dev_loss/dev_words), time.time()-start))

iter 4: dev loss/word=5.4342, ppl=229.0993, time=41.79s


# 6)-Generate a few sentences

In [20]:
# Generate a few sentences
for _ in range(5):
    sent = generate_sent()
    print(" ".join([i2w[x] for x in sent]))

in addition natural production a private company and more than anything wrong and suggest that the police whose public consequences of money investment report
separately the state and <unk> elections of in <unk> car
washington here
at home development rates
in a land television into advertisers
